In [21]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, Activation, Reshape, MaxPooling1D, GlobalAveragePooling1D
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers.convolutional import Conv1D, Conv2D
from keras import optimizers
import sys
import keras.utils
import os
sys.path.append(os.path.join(sys.path[0], '../utils'))
from load_mat_file import load_mat_file
from keras.callbacks import TensorBoard
import copy
#import matplotlib as plt

In [27]:
def create_network(input_dim, num_classes):
    '''Base network to be shared (eq. to feature extraction).
    '''
    seq = Sequential()
    seq.add(Dense(128, input_shape=(input_dim,)))
    seq.add(Activation('relu'));
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(num_classes, activation='relu'))
    model.add(Activation('softmax'))
    return seq


def create_network_tu(max_words, num_classes):
    model = Sequential()
    model.add(Dense(512, input_shape=(max_words,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model;


def compute_accuracy(predictions, labels):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < 0.5].mean()


def create_network_eeg_conv1(input_dim, num_classes):
    seq = Sequential()
    seq.add(Reshape(( input_dim, 1,), input_shape=(input_dim,)))
    seq.add(Conv1D(filters=16, kernel_size=5, activation='relu', strides=2))
    seq.add(MaxPooling1D(3, strides=2))
    seq.add(Conv1D(32, 5, activation='relu'))
    seq.add(MaxPooling1D(3, strides=2))
    seq.add(Conv1D(32, 3, activation='relu'))
    seq.add(MaxPooling1D(3, strides=2))
    seq.add(Conv1D(32, 3, activation='relu'))
    seq.add(MaxPooling1D(3, strides=2))
    seq.add(GlobalAveragePooling1D())
    #seq.add(Reshape(11328,1))
    seq.add(Dense(units=128))
    seq.add(Activation('relu'));
    seq.add(Dense(units=num_classes))
    #seq.add(Reshape((num_classes,)))
    seq.add(Activation('softmax'));
    return seq;

In [14]:
EEG_FILENAME = 'D0000168'
CHANNELS_NUM = 21
sel_channels = slice(0, CHANNELS_NUM)
sel_time = slice(None,None,1) #quanta time
EEG = load_mat_file('../../data/eegmat_selected/' + EEG_FILENAME, 's')
X_data = EEG["eeg"][0][0]
Y_data = EEG["mrk"][0][0]
#X_data = load_mat_file('../../data/test_data2', 'X');
#Y_data = load_mat_file('../../data/test_data2', 'Y');
#Y_data= np.swapaxes(Y_data, 1,0)
X_data = np.swapaxes(X_data, 2,0)
X_data = np.swapaxes(X_data, 1,2)
print('X_data_shape: ', X_data.shape)
print('Y_data_shape: ', Y_data.shape)
OneChannel_data = X_data[:,sel_time,sel_channels];
OneChannel_data = OneChannel_data.reshape(OneChannel_data.shape[0], OneChannel_data.shape[1]*OneChannel_data.shape[2])
print('X_data_reshapedshape: ', OneChannel_data.shape)
size_of_sample = OneChannel_data.shape[1];
number_of_samples = OneChannel_data.shape[0];
print('number of samples:', number_of_samples)
print('size_of_sample:', size_of_sample)
Y_data
Y_data = Y_data - 1;
Y_data = Y_data.reshape(Y_data.shape[0])
num_classes = max(Y_data) + 1;
print('num_classes: ', num_classes)
y_= keras.utils.to_categorical(Y_data, num_classes);
x_= OneChannel_data;
test_prop = 0.1;
test_size= round(test_prop * number_of_samples, 0);
test_size = int(test_size)
train_size = number_of_samples - test_size;

X_data_shape:  (180, 275, 21)
Y_data_shape:  (180, 1)
X_data_reshapedshape:  (180, 5775)
number of samples: 180
size_of_sample: 5775
num_classes:  2


In [15]:
#random shuffle of dataset and train-test split
idx = np.random.choice(np.arange(x_.shape[0]), x_.shape[0], replace=False)
idx_train = idx[0:train_size]
idx_test = idx[train_size:]
print('train size: ', train_size)
print('test size: ', test_size)
y_train = y_[idx_train];
x_train = x_[idx_train];
y_test= y_[idx_test];
x_test= x_[idx_test];

train size:  162
test size:  18


In [28]:

model = create_network_eeg_conv1(size_of_sample, num_classes)
print(model.summary())
model = create_network_tu(size_of_sample, num_classes)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_7 (Reshape)          (None, 5775, 1)           0         
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 2886, 16)          96        
_________________________________________________________________
max_pooling1d_21 (MaxPooling (None, 1442, 16)          0         
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 1438, 32)          2592      
_________________________________________________________________
max_pooling1d_22 (MaxPooling (None, 718, 32)           0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 716, 32)           3104      
_________________________________________________________________
max_pooling1d_23 (MaxPooling (None, 357, 32)           0         
__________

In [32]:
batch_size = 10;
epochs = 20;
sgd = optimizers.Adagrad(lr=0.00001, decay=1e-4)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

tensorboard=TensorBoard(log_dir='./logs', write_graph=True)

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.2,
                    callbacks=[tensorboard])

score = model.evaluate(x_test, y_test, batch_size=batch_size)
print(score)

Train on 129 samples, validate on 33 samples
Epoch 1/20
129/129 [==============================] - 1s 5ms/step - loss: 3.1209 - acc: 0.5581 - val_loss: 2.3751 - val_acc: 0.4848
Epoch 2/20
129/129 [==============================] - 0s 4ms/step - loss: 2.7081 - acc: 0.5581 - val_loss: 2.3752 - val_acc: 0.5152
Epoch 3/20
129/129 [==============================] - 0s 4ms/step - loss: 2.9034 - acc: 0.6047 - val_loss: 2.3605 - val_acc: 0.5152
Epoch 4/20
129/129 [==============================] - 0s 4ms/step - loss: 2.8124 - acc: 0.5194 - val_loss: 2.3597 - val_acc: 0.5152
Epoch 5/20
129/129 [==============================] - 0s 4ms/step - loss: 2.8153 - acc: 0.5814 - val_loss: 2.3506 - val_acc: 0.5152
Epoch 6/20
129/129 [==============================] - 1s 4ms/step - loss: 2.6387 - acc: 0.6047 - val_loss: 2.3488 - val_acc: 0.5152
Epoch 7/20
129/129 [==============================] - 1s 4ms/step - loss: 3.3609 - acc: 0.5426 - val_loss: 2.3366 - val_acc: 0.5152
Epoch 8/20
129/129 [===========